# 3. Data Updates

## 3.1. Rearrange Dataframe

In [ ]:
oneday_desktop = pd.read_excel("oneday.xlsx", "desktop")
oneday_mobile = pd.read_excel("oneday.xlsx", "mobile")
oneday_tablet = pd.read_excel("oneday.xlsx", "tablet")
period_desktop = pd.read_excel("period.xlsx", "desktop")
period_mobile = pd.read_excel("period.xlsx", "mobile")
period_tablet = pd.read_excel("period.xlsx", "tablet")

In [ ]:
def df_toint(df, date_type):
    df = df.drop(["deviceCategory"], axis=0)
    df = df.apply(pd.to_numeric)
    cols = []
    for col in df.columns:
        col = col.replace("X", "")
        col = col.split(".")
        if date_type == "oneday":
            col = col[0] + "/" + col[1] + "/" + col[2] 
        elif date_type == "period":
            col = col[0] + "/" + col[1] + "/" + col[2] + "-" + col[3] + "/" + col[4] + "/" + col[5]            
        cols.append(col)
    df.columns = cols
    return df

In [ ]:
oneday_dfs = [oneday_desktop, oneday_mobile, oneday_tablet]
oneday_desktop, oneday_mobile, oneday_tablet = [df_toint(oneday_df, "oneday") for oneday_df in oneday_dfs]
period_dfs = [period_desktop, period_mobile, period_tablet]
period_desktop, period_mobile, period_tablet = [df_toint(period_df, "period") for period_df in period_dfs]

## 3.2. Update Excel with Extracted Data

In [ ]:
# Run excel in the background.
xw.App().visible = False

In [ ]:
# Detemine which workbook to activate.
wb = xw.Book("performance.xlsx")

In [ ]:
# Detemine which worksheet to activate.
sheet_ga = wb.sheets["ga"]

In [ ]:
# Find the last column of dataframe which is to be written to Excel. 
oneday_col = sheet_ga.api.UsedRange.Find(oneday_tablet.columns[-1]).address
oneday_col = oneday_col.split('$')[1]
period_col = sheet_ga.api.UsedRange.Find(period_tablet.columns[-1]).address
period_col = period_col.split('$')[1]

In [ ]:
# Specify where to insert the extracted data.
def insert_value(sheet_ga, df, col, ga_dict):
    for key in ga_dict:
        sheet_ga.range(col + str(key)).value = df.iloc[:,-1][ga_dict[key]]  
    return sheet_ga

In [ ]:
ga_dict_desktop = {17:"users", 18:"sessions", 19:"pageviews", 20:"totalEvents", 21:"searchSessions", 22:"campaignPageviews"}
ga_dict_mobile = {27:"users", 28:"sessions", 29:"pageviews", 30:"totalEvents", 31:"searchSessions", 32:"campaignPageviews"}
ga_dict_tablet = {37:"users", 38:"sessions", 39:"pageviews", 40:"totalEvents", 41:"searchSessions", 42:"campaignPageviews"}

In [ ]:
sheet_ga = insert_value(sheet_ga, oneday_desktop, oneday_col, ga_dict_desktop)
sheet_ga = insert_value(sheet_ga, oneday_mobile, oneday_col, ga_dict_mobile)
sheet_ga = insert_value(sheet_ga, oneday_tablet, oneday_col, ga_dict_tablet)
sheet_ga = insert_value(sheet_ga, period_desktop, period_col, ga_dict_desktop)
sheet_ga = insert_value(sheet_ga, period_mobile, period_col, ga_dict_mobile)
sheet_ga = insert_value(sheet_ga, period_tablet, period_col, ga_dict_tablet)

In [ ]:
# Save and close the workbook.
wb.save()
wb.close()